# Mạng nơ-ron hồi quy

Trong module trước, chúng ta đã sử dụng các biểu diễn ngữ nghĩa phong phú của văn bản và một bộ phân loại tuyến tính đơn giản trên các embeddings. Kiến trúc này giúp nắm bắt ý nghĩa tổng hợp của các từ trong một câu, nhưng nó không tính đến **thứ tự** của các từ, vì thao tác tổng hợp trên các embeddings đã loại bỏ thông tin này từ văn bản gốc. Do các mô hình này không thể mô hình hóa thứ tự từ, chúng không thể giải quyết các nhiệm vụ phức tạp hoặc mơ hồ hơn như tạo văn bản hoặc trả lời câu hỏi.

Để nắm bắt ý nghĩa của chuỗi văn bản, chúng ta cần sử dụng một kiến trúc mạng nơ-ron khác, được gọi là **mạng nơ-ron hồi quy**, hay RNN. Trong RNN, chúng ta đưa câu qua mạng từng ký hiệu một lần, và mạng tạo ra một **trạng thái**, sau đó chúng ta đưa trạng thái này trở lại mạng cùng với ký hiệu tiếp theo.

Với chuỗi đầu vào các token $X_0,\dots,X_n$, RNN tạo ra một chuỗi các khối mạng nơ-ron và huấn luyện chuỗi này từ đầu đến cuối bằng cách sử dụng lan truyền ngược. Mỗi khối mạng nhận một cặp $(X_i,S_i)$ làm đầu vào và tạo ra $S_{i+1}$ làm kết quả. Trạng thái cuối cùng $S_n$ hoặc đầu ra $X_n$ được đưa vào một bộ phân loại tuyến tính để tạo ra kết quả. Tất cả các khối mạng đều chia sẻ cùng một trọng số và được huấn luyện từ đầu đến cuối chỉ với một lần lan truyền ngược.

Vì các vector trạng thái $S_0,\dots,S_n$ được truyền qua mạng, nó có khả năng học các phụ thuộc tuần tự giữa các từ. Ví dụ, khi từ *not* xuất hiện ở đâu đó trong chuỗi, mạng có thể học cách phủ định một số phần tử trong vector trạng thái, dẫn đến phủ định.

> Vì các trọng số của tất cả các khối RNN trong hình đều được chia sẻ, hình ảnh này có thể được biểu diễn dưới dạng một khối duy nhất (bên phải) với một vòng lặp hồi quy, nơi trạng thái đầu ra của mạng được truyền lại vào đầu vào.

Hãy cùng xem cách mạng nơ-ron hồi quy có thể giúp chúng ta phân loại tập dữ liệu tin tức.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Bộ phân loại RNN đơn giản

Trong trường hợp RNN đơn giản, mỗi đơn vị hồi tiếp là một mạng tuyến tính đơn giản, nhận một vector đầu vào được nối và một vector trạng thái, sau đó tạo ra một vector trạng thái mới. PyTorch biểu diễn đơn vị này bằng lớp `RNNCell`, và một mạng lưới các đơn vị như vậy - bằng lớp `RNN`.

Để định nghĩa một bộ phân loại RNN, trước tiên chúng ta sẽ áp dụng một lớp nhúng để giảm chiều của từ vựng đầu vào, sau đó sử dụng lớp RNN ở phía trên:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Chúng ta sử dụng lớp nhúng chưa được huấn luyện ở đây để đơn giản hóa, nhưng để đạt kết quả tốt hơn, chúng ta có thể sử dụng lớp nhúng đã được huấn luyện trước với Word2Vec hoặc GloVe embeddings, như đã được mô tả trong bài học trước. Để hiểu rõ hơn, bạn có thể điều chỉnh mã này để hoạt động với các nhúng đã được huấn luyện trước.

Trong trường hợp của chúng ta, chúng ta sẽ sử dụng bộ tải dữ liệu được đệm, vì vậy mỗi batch sẽ có một số chuỗi được đệm với cùng độ dài. Lớp RNN sẽ nhận chuỗi các tensor nhúng và tạo ra hai đầu ra:
* $x$ là chuỗi các đầu ra của cell RNN tại mỗi bước
* $h$ là trạng thái ẩn cuối cùng cho phần tử cuối cùng của chuỗi

Sau đó, chúng ta áp dụng một bộ phân loại tuyến tính kết nối đầy đủ để lấy số lượng lớp.

> **Note:** RNN khá khó huấn luyện, bởi vì khi các cell RNN được mở rộng theo chiều dài chuỗi, số lượng lớp liên quan đến quá trình lan truyền ngược sẽ rất lớn. Do đó, chúng ta cần chọn tốc độ học nhỏ và huấn luyện mạng trên tập dữ liệu lớn hơn để đạt kết quả tốt. Quá trình này có thể mất khá nhiều thời gian, vì vậy việc sử dụng GPU được ưu tiên.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Bộ nhớ ngắn dài (LSTM)

Một trong những vấn đề chính của RNN cổ điển là vấn đề **độ dốc biến mất**. Vì RNN được huấn luyện từ đầu đến cuối trong một lần truyền ngược, nên rất khó để truyền lỗi đến các lớp đầu tiên của mạng, dẫn đến mạng không thể học được mối quan hệ giữa các token xa nhau. Một trong những cách để tránh vấn đề này là giới thiệu **quản lý trạng thái rõ ràng** bằng cách sử dụng các **cổng**. Có hai kiến trúc nổi tiếng nhất thuộc loại này: **Bộ nhớ ngắn dài** (LSTM) và **Đơn vị chuyển tiếp có cổng** (GRU).

![Hình ảnh minh họa một tế bào bộ nhớ ngắn dài](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Mạng LSTM được tổ chức theo cách tương tự như RNN, nhưng có hai trạng thái được truyền từ lớp này sang lớp khác: trạng thái thực tế $c$, và vector ẩn $h$. Tại mỗi đơn vị, vector ẩn $h_i$ được nối với đầu vào $x_i$, và chúng kiểm soát những gì xảy ra với trạng thái $c$ thông qua các **cổng**. Mỗi cổng là một mạng nơ-ron với hàm kích hoạt sigmoid (đầu ra trong phạm vi $[0,1]$), có thể được coi như một mặt nạ bit khi nhân với vector trạng thái. Có các cổng sau đây (từ trái sang phải trên hình ảnh ở trên):
* **Cổng quên** lấy vector ẩn và xác định những thành phần nào của vector $c$ cần quên, và những thành phần nào cần giữ lại.
* **Cổng đầu vào** lấy một số thông tin từ đầu vào và vector ẩn, rồi chèn nó vào trạng thái.
* **Cổng đầu ra** biến đổi trạng thái thông qua một lớp tuyến tính với kích hoạt $\tanh$, sau đó chọn một số thành phần của nó bằng cách sử dụng vector ẩn $h_i$ để tạo ra trạng thái mới $c_{i+1}$.

Các thành phần của trạng thái $c$ có thể được coi như các cờ hiệu có thể bật hoặc tắt. Ví dụ, khi chúng ta gặp một cái tên *Alice* trong chuỗi, chúng ta có thể giả định rằng nó đề cập đến một nhân vật nữ, và bật cờ trong trạng thái rằng chúng ta có danh từ nữ trong câu. Khi chúng ta gặp thêm cụm từ *và Tom*, chúng ta sẽ bật cờ rằng chúng ta có danh từ số nhiều. Do đó, bằng cách thao tác trạng thái, chúng ta có thể theo dõi các thuộc tính ngữ pháp của các phần trong câu.

> **Note**: Một tài liệu tuyệt vời để hiểu rõ nội bộ của LSTM là bài viết [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) của Christopher Olah.

Mặc dù cấu trúc nội bộ của tế bào LSTM có thể trông phức tạp, PyTorch đã ẩn việc triển khai này bên trong lớp `LSTMCell`, và cung cấp đối tượng `LSTM` để đại diện cho toàn bộ lớp LSTM. Do đó, việc triển khai bộ phân loại LSTM sẽ khá giống với RNN đơn giản mà chúng ta đã thấy ở trên:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Chuỗi được đóng gói

Trong ví dụ của chúng ta, chúng ta đã phải thêm các vector số 0 để làm đầy tất cả các chuỗi trong minibatch. Mặc dù điều này dẫn đến việc lãng phí bộ nhớ, nhưng với RNN, vấn đề quan trọng hơn là các tế bào RNN bổ sung được tạo ra cho các phần tử đầu vào đã được làm đầy, những phần tử này tham gia vào quá trình huấn luyện nhưng không mang thông tin đầu vào quan trọng. Sẽ tốt hơn nhiều nếu chỉ huấn luyện RNN với kích thước chuỗi thực tế.

Để làm điều đó, một định dạng đặc biệt để lưu trữ chuỗi đã làm đầy được giới thiệu trong PyTorch. Giả sử chúng ta có một minibatch đầu vào đã làm đầy trông như thế này:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Ở đây, số 0 đại diện cho các giá trị làm đầy, và vector độ dài thực tế của các chuỗi đầu vào là `[5,3,1]`.

Để huấn luyện RNN hiệu quả với chuỗi đã làm đầy, chúng ta muốn bắt đầu huấn luyện nhóm đầu tiên của các tế bào RNN với minibatch lớn (`[1,6,9]`), sau đó kết thúc xử lý chuỗi thứ ba, và tiếp tục huấn luyện với các minibatch nhỏ hơn (`[2,7]`, `[3,8]`), và cứ thế. Vì vậy, chuỗi được đóng gói được biểu diễn dưới dạng một vector - trong trường hợp của chúng ta là `[1,6,9,2,7,3,8,4,5]`, và vector độ dài (`[5,3,1]`), từ đó chúng ta có thể dễ dàng tái tạo lại minibatch đã làm đầy ban đầu.

Để tạo chuỗi được đóng gói, chúng ta có thể sử dụng hàm `torch.nn.utils.rnn.pack_padded_sequence`. Tất cả các lớp hồi quy, bao gồm RNN, LSTM và GRU, đều hỗ trợ chuỗi được đóng gói làm đầu vào, và tạo ra đầu ra được đóng gói, có thể được giải mã bằng cách sử dụng `torch.nn.utils.rnn.pad_packed_sequence`.

Để có thể tạo chuỗi được đóng gói, chúng ta cần truyền vector độ dài vào mạng, và do đó chúng ta cần một hàm khác để chuẩn bị các minibatch:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Mạng thực tế sẽ rất giống với `LSTMClassifier` ở trên, nhưng bước `forward` sẽ nhận cả minibatch đã được đệm và vector độ dài của các chuỗi. Sau khi tính embedding, chúng ta tính packed sequence, truyền nó qua lớp LSTM, và sau đó giải nén kết quả trở lại.

> **Lưu ý**: Thực tế, chúng ta không sử dụng kết quả đã được giải nén `x`, vì chúng ta sử dụng đầu ra từ các lớp ẩn trong các phép tính tiếp theo. Do đó, chúng ta có thể loại bỏ hoàn toàn bước giải nén khỏi đoạn mã này. Lý do chúng tôi đặt nó ở đây là để bạn có thể dễ dàng chỉnh sửa đoạn mã này, trong trường hợp bạn cần sử dụng đầu ra của mạng trong các phép tính tiếp theo.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Lưu ý:** Bạn có thể đã nhận thấy tham số `use_pack_sequence` mà chúng ta truyền vào hàm huấn luyện. Hiện tại, hàm `pack_padded_sequence` yêu cầu tensor độ dài chuỗi phải nằm trên thiết bị CPU, và do đó hàm huấn luyện cần tránh di chuyển dữ liệu độ dài chuỗi sang GPU khi huấn luyện. Bạn có thể xem xét việc triển khai hàm `train_emb` trong tệp [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## RNN hai chiều và nhiều lớp

Trong các ví dụ của chúng ta, tất cả các mạng hồi quy đều hoạt động theo một hướng, từ đầu chuỗi đến cuối chuỗi. Điều này có vẻ tự nhiên, vì nó giống cách chúng ta đọc và nghe lời nói. Tuy nhiên, trong nhiều trường hợp thực tế, chúng ta có thể truy cập ngẫu nhiên vào chuỗi đầu vào, nên việc thực hiện tính toán hồi quy theo cả hai hướng có thể hợp lý. Các mạng như vậy được gọi là **RNN hai chiều**, và chúng có thể được tạo bằng cách truyền tham số `bidirectional=True` vào hàm khởi tạo của RNN/LSTM/GRU.

Khi làm việc với mạng hai chiều, chúng ta sẽ cần hai vector trạng thái ẩn, một cho mỗi hướng. PyTorch mã hóa các vector này thành một vector có kích thước gấp đôi, điều này khá tiện lợi, vì thông thường bạn sẽ truyền trạng thái ẩn kết quả vào lớp tuyến tính kết nối đầy đủ, và bạn chỉ cần tính đến sự gia tăng kích thước này khi tạo lớp.

Mạng hồi quy, dù một chiều hay hai chiều, đều nắm bắt các mẫu nhất định trong một chuỗi và có thể lưu trữ chúng vào vector trạng thái hoặc truyền vào đầu ra. Tương tự như mạng tích chập, chúng ta có thể xây dựng một lớp hồi quy khác trên lớp đầu tiên để nắm bắt các mẫu cấp cao hơn, được xây dựng từ các mẫu cấp thấp do lớp đầu tiên trích xuất. Điều này dẫn đến khái niệm **RNN nhiều lớp**, bao gồm hai hoặc nhiều mạng hồi quy, trong đó đầu ra của lớp trước được truyền vào lớp tiếp theo làm đầu vào.

![Hình ảnh minh họa một mạng RNN LSTM nhiều lớp](../../../../../translated_images/vi/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Hình ảnh từ [bài viết tuyệt vời này](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) của Fernando López*

PyTorch giúp việc xây dựng các mạng như vậy trở nên dễ dàng, vì bạn chỉ cần truyền tham số `num_layers` vào hàm khởi tạo của RNN/LSTM/GRU để tự động tạo nhiều lớp hồi quy. Điều này cũng có nghĩa là kích thước của vector trạng thái ẩn sẽ tăng lên tương ứng, và bạn cần tính đến điều này khi xử lý đầu ra của các lớp hồi quy.


## RNNs cho các nhiệm vụ khác

Trong bài học này, chúng ta đã thấy rằng RNNs có thể được sử dụng để phân loại chuỗi, nhưng thực tế, chúng có thể xử lý nhiều nhiệm vụ khác như tạo văn bản, dịch máy, và nhiều hơn nữa. Chúng ta sẽ xem xét những nhiệm vụ đó trong bài học tiếp theo.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
